In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key="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"
)

In [4]:
response = llm.invoke("What is the capital of France?")
response

AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 42, 'total_tokens': 50, 'completion_time': 0.032, 'prompt_time': 0.011044119, 'queue_time': 0.005382427, 'total_time': 0.043044119}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_5c5d1b5cfb', 'finish_reason': 'stop', 'logprobs': None}, id='run-cdd0cce9-5e37-4c23-bb04-b91617f41864-0', usage_metadata={'input_tokens': 42, 'output_tokens': 8, 'total_tokens': 50})

In [13]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-38844?from=job%20search%20funnel")

page_data = loader.load().pop().page_content

page_data

'Apply for Software Engineer I\n\nSearch JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer IBeaverton, OregonBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does more t

In [14]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
res

AIMessage(content='```json\n{\n  "role": "Software Engineer I",\n  "experience": "2+ years of professional experience in application engineering",\n  "skills": [\n    "JavaScript (Node.js, React.js, Vue.js, etc.)",\n    "Agile processes",\n    "Continuous integration",\n    "Test automation",\n    "AWS CloudFront",\n    "AWS S3",\n    "AWS Route 53",\n    "AWS Certificate Manager",\n    "AWS Lambda",\n    "GitHub Co-pilot"\n  ],\n  "description": "The Digital Employee Experience Software Engineer I will be responsible for design, support, development, and testing of internal employee-facing websites and web applications. Design and develop technical solutions that meet the needs of the business and aligns with Nike architectural and quality standards. Build mobile first Front-End applications in a manner visually and culturally consistent with Nike brand using React.js and Redux. Ensure SLAs (Service Level Agreements) for different interfaces are met and test them by executing performa

In [21]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()

json_response = json_parser.parse(res.content)

json_response

{'role': 'Software Engineer I',
 'experience': '2+ years of professional experience in application engineering',
 'skills': ['JavaScript (Node.js, React.js, Vue.js, etc.)',
  'Agile processes',
  'Continuous integration',
  'Test automation',
  'AWS CloudFront',
  'AWS S3',
  'AWS Route 53',
  'AWS Certificate Manager',
  'AWS Lambda',
  'GitHub Co-pilot'],
 'description': 'The Digital Employee Experience Software Engineer I will be responsible for design, support, development, and testing of internal employee-facing websites and web applications. Design and develop technical solutions that meet the needs of the business and aligns with Nike architectural and quality standards. Build mobile first Front-End applications in a manner visually and culturally consistent with Nike brand using React.js and Redux. Ensure SLAs (Service Level Agreements) for different interfaces are met and test them by executing performance tests using various loads. Provide production support on new and existi

In [22]:
import pandas as pd

df = pd.read_csv('my_portfolio.csv')

df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [24]:
import chromadb

import uuid

client = chromadb.PersistentClient('vectorstore')

collection = client.get_or_create_collection('portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(
            documents = row["Techstack"],
            metadatas = {"links" : row["Links"]},
            ids = [str(uuid.uuid4())]
        )

C:\Users\qasim\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [03:43<00:00, 372kiB/s] 


In [28]:
links = collection.query(query_texts=json_response['skills'], n_results=2).get('metadatas', [])

links

[[{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/react-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],


In [29]:
job = json_response
job['skills']

['JavaScript (Node.js, React.js, Vue.js, etc.)',
 'Agile processes',
 'Continuous integration',
 'Test automation',
 'AWS CloudFront',
 'AWS S3',
 'AWS Route 53',
 'AWS Certificate Manager',
 'AWS Lambda',
 'GitHub Co-pilot']

In [30]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Solutions for Nike's Digital Employee Experience

Dear Hiring Manager,

I came across the job description for a Software Engineer I position at Nike, and I'm excited to introduce AtliQ, an AI & Software Consulting company that can help you achieve your goals. With our expertise in application engineering, we can support the design, development, and testing of internal employee-facing websites and web applications.

Our team has extensive experience in JavaScript (Node.js, React.js, Vue.js, etc.), Agile processes, Continuous integration, Test automation, and AWS services (CloudFront, S3, Route 53, Certificate Manager, and Lambda). We're confident that our skills align with Nike's architectural and quality standards.

We've worked on numerous projects that demonstrate our capabilities in building mobile-first Front-End applications using React.js and Redux. Our expertise in performance testing and ensuring SLAs (Service Level Agreements) can help you 